In [11]:
!pip install open3d matplotlib
!pip install trimesh


In [1]:
pose_file="/home/koerner/Project/nice-slam/Datasets/gt_own/room0/traj.txt"
ply_file="/home/koerner/Project/nice-slam/Datasets/Replica/room0_mesh.ply"

In [2]:
import trimesh
ply_file = '/home/koerner/Project/nice-slam/Datasets/Replica/room0_mesh.ply'  # Update with your path

try:
    mesh = trimesh.load(ply_file)
    # Continue with your processing...
except Exception as e:
    print(f"Error loading mesh: {e}")

In [4]:


# Load the mesh using trimesh
ply_file = ply_file  # Replace with your .ply file path
mesh = trimesh.load(ply_file)
vertices = np.asarray(mesh.vertices)
faces = np.asarray(mesh.faces)

# Load your 4x4 matrices representing the trajectory
pose_file =pose_file  # Replace with your pose file path
rotation_matrix_x= np.array([[1, 0, 0, 0],
                                [0, 0, -1, 0],
                                [0, 1, 0, 0],
                                [0, 0, 0, 1]])
T_wc = np.loadtxt(pose_file).reshape(-1, 4, 4)
T_wc = rotation_matrix_x@T_wc
trajectories = [T_wc[i] for i in range(T_wc.shape[0])]

# Set up the plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot the mesh (simple wireframe)
ax.plot_trisurf(vertices[:, 0], vertices[:, 1], vertices[:, 2], triangles=faces, linewidth=0.2, antialiased=True)

for idx, mat in enumerate(trajectories):
    # Extract camera position from the matrix
    camera_position = mat[:3, 3]
    ax.axis('off')
    # Set the camera position
    ax.view_init(azim=camera_position[0], elev=camera_position[1])

    # Save the image
    plt.savefig(f'image_{idx}.png')

plt.close()


KeyboardInterrupt: 

Error in callback <function flush_figures at 0x7f27ad106790> (for post_execute):


KeyboardInterrupt: 

In [6]:
!pip install pyrender

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 26.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 858.3/858.3 kB 7.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.7 MB/s eta 0:00:00a 0:00:01
  Created wheel for PyOpenGL: filename=PyOpenGL-3.1.0-py3-none-any.whl size=1745192 sha256=14358d7eebd10eda583187ff3ea0fd7d4959f7495b4e5f7714a9dd5b7bc407a4
  Stored in directory: /home/koerner/.cache/pip/wheels/ff/a0/d3/f562b4c97f9b23a5c28dc55a3586a2f5c1d8af766cd8d9bd3a
Successfully built PyOpenGL


In [5]:
import os
os.environ['PYOPENGL_PLATFORM'] = 'egl'
import trimesh
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

In [6]:
import trimesh
import pyrender
fuze_trimesh = trimesh.load('/home/koerner/Project/nice-slam/Datasets/Replica/room0_mesh.ply')
mesh = pyrender.Mesh.from_trimesh(fuze_trimesh)
scene = pyrender.Scene()
scene.add(mesh)
camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0, aspectRatio=1.0)
s = np.sqrt(2)/2
camera_pose = trajectories[0]
scene.add(camera, pose=camera_pose)
light = pyrender.SpotLight(color=np.ones(3), intensity=3.0,
                          innerConeAngle=np.pi/16.0,
                          outerConeAngle=np.pi/6.0)
scene.add(light, pose=camera_pose)
r = pyrender.OffscreenRenderer(400, 400)
color, depth = r.render(scene)
plt.figure()
plt.subplot(1,2,1)
plt.axis('off')
plt.imshow(color)
plt.subplot(1,2,2)
plt.axis('off')
plt.imshow(depth, cmap=plt.cm.gray_r)
plt.show()

NameError: name 'trajectories' is not defined